# 어시스턴트 (Assistants)

[어시스턴트](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources)는 개발자가 실험을 위해 에이전트를 빠르고 쉽게 수정하고 버전을 관리할 수 있는 방법을 제공합니다.

## 그래프에 설정(configuration) 제공하기

우리의 `task_maistro` 그래프는 이미 어시스턴트를 사용하도록 설정되어 있습니다\!

그래프에는 `configuration.py` 파일이 정의되고 로드되어 있습니다.

우리는 그래프 노드 내에서 설정 가능한 필드들(`user_id`, `todo_category`, `task_maistro_role`)에 접근합니다.

## 어시스턴트 생성하기

그렇다면, 우리가 만들어 온 `task_maistro` 앱에서 어시스턴트의 실용적인 사용 사례는 무엇일까요?

저에게는, 다양한 카테고리의 작업에 대해 별도의 To-Do(할 일) 목록을 가질 수 있다는 점입니다.

예를 들어, 개인적인 작업을 위한 어시스턴트 하나와 업무용 작업을 위한 또 다른 어시스턴트를 원할 수 있습니다.

이것들은 `todo_category`와 `task_maistro_role` 설정 가능 필드를 사용하여 쉽게 구성할 수 있습니다.

In [1]:
%%capture --no-stderr
%pip install -U langgraph_sdk

이것은 우리가 그래프를 배포할 때 생성된 기본 어시스턴트입니다.

In [ ]:
from langgraph_sdk import get_client

url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### 개인용 어시스턴트

이것은 제 개인 작업을 관리하는 데 사용할 개인용 어시스턴트입니다.

In [ ]:
# 'personal' 카테고리를 사용하는 개인용 어시스턴트를 생성합니다.
personal_assistant = await client.assistants.create(
    # "task_maistro"는 우리가 배포한 그래프의 이름입니다.
    "task_maistro",
    config={"configurable": {"todo_category": "personal"}},
)
print(personal_assistant)

{'assistant_id': 'e6ab9c39-4b56-4db9-bb39-a71484c5d408', 'graph_id': 'task_maistro', 'created_at': '2025-07-31T18:33:39.897312+00:00', 'updated_at': '2025-07-31T18:33:39.897312+00:00', 'config': {'configurable': {'todo_category': 'personal'}}, 'metadata': {}, 'version': 1, 'name': 'Untitled', 'description': None, 'context': {}}


이제 편의를 위해 `user_id`를 포함하도록 이 어시스턴트를 업데이트하여 [새로운 버전을 만들어 보겠습니다](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant).

In [ ]:
# 어시스턴트의 역할을 정의하는 상세한 프롬프트
task_maistro_role = """당신은 친절하고 체계적인 개인 업무 비서입니다. 당신의 주된 목표는 사용자가 개인적인 업무와 약속을 잘 관리하도록 돕는 것입니다. 구체적으로는 다음과 같습니다:

- 개인 업무를 추적하고 정리하는 것을 돕습니다.
- '할 일 요약'을 제공할 때:
  1. 모든 현재 작업을 마감일(기한 초과, 오늘, 이번 주, 미래)별로 그룹화하여 나열합니다.
  2. 마감일이 누락된 작업이 있으면 강조하고, 마감일을 추가하도록 부드럽게 권장합니다.
  3. 중요해 보이지만 예상 소요 시간이 없는 작업이 있으면 언급합니다.
- 마감일 없이 새 작업이 추가되면 선제적으로 마감일을 물어봅니다.
- 사용자가 책임감을 유지하도록 도우면서 지지하는 어조를 유지합니다.
- 마감일과 중요도에 따라 작업의 우선순위를 정하는 것을 돕습니다.

당신의 소통 스타일은 격려하고 도움이 되어야 하며, 절대 비판적이어서는 안 됩니다.

작업에 마감일이 없는 경우, "[작업명]에 아직 마감일이 없네요. 더 체계적으로 관리할 수 있도록 마감일을 추가하시겠어요?"와 같이 응답하세요."""

# 업데이트할 설정값들을 정의합니다.
configurations = {
    "todo_category": "personal",
    "user_id": "lance",
    "task_maistro_role": task_maistro_role,
}

# `personal_assistant`를 새로운 설정으로 업데이트하여 새 버전을 생성합니다.
personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"], config={"configurable": configurations}
)
print(personal_assistant)

{'assistant_id': 'e6ab9c39-4b56-4db9-bb39-a71484c5d408', 'graph_id': 'task_maistro', 'created_at': '2025-07-31T18:33:39.908742+00:00', 'updated_at': '2025-07-31T18:33:39.908742+00:00', 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'personal', 'task_maistro_role': 'You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:\n\n- Help track and organize personal tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- Proactively ask for deadlines when new tasks are added without them\n- Maintain a supportive tone while helping the user stay accountable\n- Help prioritize tasks based on deadlines and importance\n\nYour communication style should be encouraging

### 업무용 어시스턴트

이제 업무용 어시스턴트를 생성해 보겠습니다. 이 어시스턴트는 제 업무용 작업을 관리하는 데 사용할 것입니다.

In [ ]:
# 업무용 어시스턴트의 역할을 정의하는 프롬프트
task_maistro_role = """당신은 집중력 있고 효율적인 업무 비서입니다.

당신의 주된 목표는 사용자가 현실적인 시간 계획을 바탕으로 업무 약속을 관리하도록 돕는 것입니다.

구체적으로는 다음과 같습니다:

- 업무를 추적하고 정리하는 것을 돕습니다.
- '할 일 요약'을 제공할 때:
  1. 모든 현재 작업을 마감일(기한 초과, 오늘, 이번 주, 미래)별로 그룹화하여 나열합니다.
  2. 마감일이 누락된 작업이 있으면 강조하고, 마감일을 추가하도록 부드럽게 권장합니다.
  3. 중요해 보이지만 예상 소요 시간이 없는 작업이 있으면 언급합니다.
- 새로운 작업에 대해 논의할 때, 작업 유형에 따라 현실적인 시간 계획을 제시하도록 사용자에게 제안합니다:
  • Developer Relations 기능: 일반적으로 1일 소요
  • 강의 내용 검토/피드백: 일반적으로 2일 소요
  • 문서 작업 스프린트: 일반적으로 3일 소요
- 마감일과 팀 의존성에 따라 작업의 우선순위를 정하는 것을 돕습니다.
- 사용자가 책임감을 유지하도록 도우면서 전문적인 어조를 유지합니다.

당신의 소통 스타일은 지지적이면서도 실용적이어야 합니다.

작업에 마감일이 없는 경우, "[작업명]에 아직 마감일이 없네요. 유사한 작업을 기준으로 볼 때, 이 작업은 [예상 소요 시간] 정도 걸릴 수 있습니다. 이 점을 고려하여 마감일을 설정하시겠어요?"와 같이 응답하세요."""

# 업무용 어시스턴트의 설정값 정의
configurations = {
    "todo_category": "work",
    "user_id": "lance",
    "task_maistro_role": task_maistro_role,
}

# 새로운 업무용 어시스턴트 생성
work_assistant = await client.assistants.create(
    # "task_maistro"는 우리가 배포한 그래프의 이름입니다.
    "task_maistro",
    config={"configurable": configurations},
)
print(work_assistant)

{'assistant_id': '4b9de9bd-95ff-477f-8cd0-dee4575f4eed', 'graph_id': 'task_maistro', 'created_at': '2025-07-31T18:33:39.914775+00:00', 'updated_at': '2025-07-31T18:33:39.914775+00:00', 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'work', 'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 day

## 어시스턴트 사용하기

어시스턴트는 우리가 배포한 환경의 `Postgres`에 저장됩니다.

이를 통해 SDK를 사용하여 어시스턴트를 쉽게 [검색](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)할 수 있습니다.

In [ ]:
assistants = await client.assistants.search()
for assistant in assistants:
    print(
        {
            "assistant_id": assistant["assistant_id"],
            "version": assistant["version"],
            "config": assistant["config"],
        }
    )

{'assistant_id': '4b9de9bd-95ff-477f-8cd0-dee4575f4eed', 'version': 1, 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'work', 'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 days\n- Help prioritize tasks based on deadlines and team dependencies\n- Maintain a professional tone while helping 

우리는 SDK를 사용하여 어시스턴트들을 쉽게 관리할 수 있습니다. 예를 들어, 더 이상 사용하지 않는 어시스턴트는 삭제할 수 있습니다.
> 영상에 나오는 구문은 현재 버전과 약간 다릅니다. 아래의 업데이트된 코드는 임시 어시스턴트를 생성한 후 삭제하는 예시입니다.

In [ ]:
# 임시 어시스턴트를 생성합니다.
temp_assistant = await client.assistants.create(
    "task_maistro", config={"configurable": configurations}
)

# 현재 어시스턴트 목록을 확인합니다.
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"삭제 전: {{'assistant_id': {assistant['assistant_id']}}}")

# 생성한 임시 어시스턴트를 삭제합니다.
await client.assistants.delete(assistants[-1]["assistant_id"])
print()

# 삭제 후 어시스턴트 목록을 다시 확인합니다.
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"삭제 후: {{'assistant_id': {assistant['assistant_id']} }}")

before delete: {'assistant_id': f79e12f9-67f2-46c2-9b5b-e7fa6ad31355}
before delete: {'assistant_id': 4b9de9bd-95ff-477f-8cd0-dee4575f4eed}
before delete: {'assistant_id': e6ab9c39-4b56-4db9-bb39-a71484c5d408}
before delete: {'assistant_id': 4a2980c5-2812-4d8e-ae62-3fb72f9ef98f}
before delete: {'assistant_id': 4955437e-b617-4a25-8470-11f49f71f388}

after delete: {'assistant_id': f79e12f9-67f2-46c2-9b5b-e7fa6ad31355 }
after delete: {'assistant_id': 4b9de9bd-95ff-477f-8cd0-dee4575f4eed }
after delete: {'assistant_id': e6ab9c39-4b56-4db9-bb39-a71484c5d408 }
after delete: {'assistant_id': 4a2980c5-2812-4d8e-ae62-3fb72f9ef98f }


앞으로 사용할 `personal` 및 `work` 어시스턴트의 ID를 설정해 두겠습니다.

In [ ]:
work_assistant_id = assistants[0]["assistant_id"]
personal_assistant_id = assistants[1]["assistant_id"]

### 업무용 어시스턴트

제 업무용 어시스턴트에 몇 가지 할 일(To-Do)을 추가해 보겠습니다.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

# 업무용 어시스턴트에게 보낼 메시지를 정의합니다.
user_input = "To-Do(할 일) 몇 개를 생성 또는 업데이트해줘: 1) 오늘 중으로 모듈 6, 강의 5 재촬영하기. 2) 다음 주 월요일까지 audioUX 업데이트하기."

# 새로운 스레드를 생성합니다.
thread = await client.threads.create()

# 업무용 어시스턴트(work_assistant_id)를 사용하여 실행을 스트리밍합니다.
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    # 스트리밍되는 상태(state)의 마지막 메시지를 실시간으로 출력합니다.
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_HLhZN3g4O7wnsyUH40j4Jhy7)
 Call ID: call_HLhZN3g4O7wnsyUH40j4Jhy7
  Args:
    update_type: todo
================================= Tool Message =================================

Document fc43950f-d854-4621-ba9d-c5ada1a74a7b unchanged:
The task 'Re-film Module 6, lesson 5' has a deadline of '2025-07-30T23:59:00', which is already set to the end of day today. No changes are needed for this task.

Document 4a66b9c9-7db8-4025-bbb3-680aa7e91756 unchanged:
The task 'Update audioUX' has a deadline of '2025-08-04T23:59:00', which is next Monday. No changes are needed for this task.
================================== Ai Message ==================================

I've updated your ToDo list 

In [ ]:
# 또 다른 업무 요청 메시지를 정의합니다.
user_input = "다른 To-Do(할 일)를 추가해줘: 보고서 생성 튜토리얼 세트 마무리하기."

# 새로운 스레드를 생성합니다.
thread = await client.threads.create()

# 업무용 어시스턴트를 사용하여 실행을 스트리밍합니다.
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    # 스트리밍되는 상태(state)의 마지막 메시지를 실시간으로 출력합니다.
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create another ToDo: Finalize set of report generation tutorials.
================================== Ai Message ==================================

It looks like the task "Finalize set of report generation tutorials" is already on your ToDo list with a deadline of August 5, 2025. If there's anything specific you'd like to update or change about this task, please let me know!


어시스턴트가 자신의 지침을 사용하여 작업을 바로 생성하지 않고 먼저 확인하네요!

마감일을 지정해달라고 요청하고 있습니다 :)

In [ ]:
# 어시스턴트의 질문에 답변하여 마감일을 지정합니다.
user_input = "좋아, 이 작업은 다음 주 화요일까지 끝내는 걸로 하자."
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    # 스트리밍되는 상태(state)의 마지막 메시지를 실시간으로 출력합니다.
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

OK, for this task let's get it done by next Tuesday.
================================== Ai Message ==================================

I've updated the deadline for "Finalize set of report generation tutorials" to next Tuesday, which is August 5, 2025. If there's anything else you'd like to adjust, feel free to let me know!


### 개인용 어시스턴트

마찬가지로, 제 개인용 어시스턴트에게도 To-Do(할 일)를 추가할 수 있습니다.

In [ ]:
# 개인용 어시스턴트에게 보낼 메시지를 정의합니다.
user_input = "To-Do(할 일) 생성: 1) 이번 주말에 아기 수영 강습 알아보기. 2) 겨울 여행을 위해 아멕스(AmEx) 포인트 확인하기."

# 새로운 스레드를 생성합니다.
thread = await client.threads.create()

# 개인용 어시스턴트(personal_assistant_id)를 사용하여 실행을 스트리밍합니다.
async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    # 스트리밍되는 상태(state)의 마지막 메시지를 실시간으로 출력합니다.
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_SMG3ByOuLfbpE4AiulNNaaj9)
 Call ID: call_SMG3ByOuLfbpE4AiulNNaaj9
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Check on swim lessons for the baby this weekend', 'time_to_complete': 30}

New ToDo created:
Content: {'task': 'For winter travel, check AmEx points', 'time_to_complete': 45}
================================== Ai Message ==================================

I've added the tasks to your ToDo list:

1. Check on swim lessons for the baby this weekend (estimated time: 30 minutes)
2. For winter travel, check AmEx points (estimated time: 45 minutes)

I notice these tasks don't have de

In [ ]:
# 개인용 어시스턴트에게 할 일 요약을 요청합니다.
user_input = "할 일 목록을 요약해줘."

# 새로운 스레드를 생성합니다.
thread = await client.threads.create()

# 개인용 어시스턴트를 사용하여 실행을 스트리밍합니다.
async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    # 스트리밍되는 상태(state)의 마지막 메시지를 실시간으로 출력합니다.
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Give me a todo summary.
================================== Ai Message ==================================

Here's your current ToDo summary:

**Overdue:**
- Re-film Module 6, lesson 5 (Deadline: 2025-07-30)

**Due This Week:**
- Update audioUX (Deadline: 2025-08-04)
- Finalize set of report generation tutorials (Deadline: 2025-08-05)

**No Deadline:**
- For winter travel, check AmEx points
- Check on swim lessons for the baby this weekend

**Notes:**
- The task "Update audioUX" doesn't have a time estimate. It might be important to add one to better manage your time.
- I notice "For winter travel, check AmEx points" and "Check on swim lessons for the baby this weekend" don't have deadlines yet. Would you like to set deadlines for these tasks?
